In [0]:
!pip3 install http://download.pytorch.org/whl/cu90/torch-1.0.0-cp36-cp36m-linux_x86_64.whl
!pip3 install torchvision
!pip3 install tqdm

In [0]:
emb = nn.Embedding(10000, 20, padding_idx=0)
# Embedding 계층의 입력은 int64 Tensor
inp = torch.tensor([1, 2, 5, 2, 10], dtype=torch.int64)
# 출력은 float32 Tensor
out = emb(inp)


In [0]:
!wget http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar xf aclImdb_v1.tar.gz


--2018-12-14 09:30:27--  http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
Resolving ai.stanford.edu (ai.stanford.edu)... 171.64.68.10
Connecting to ai.stanford.edu (ai.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 84125825 (80M) [application/x-gzip]
Saving to: ‘aclImdb_v1.tar.gz’

aclImdb_v1.tar.gz   100%[===================>]  80.23M  19.9MB/s    in 6.5s    

2018-12-14 09:30:34 (12.4 MB/s) - ‘aclImdb_v1.tar.gz’ saved [84125825/84125825]



In [0]:
import glob
import pathlib
import re

remove_marks_regex = re.compile("[,\.\(\)\[\]\*:;]|<.*?>")
shift_marks_regex = re.compile("([?!])")

def text2ids(text, vocab_dict):
    # !? 이외의 기호 삭제
    text = remove_marks_regex.sub("", text)
    # !?와 단어 사이에 공백 삽입
    text = shift_marks_regex.sub(r" \1 ", text)
    tokens = text.split()
    return [vocab_dict.get(token, 0) for token in tokens]

def list2tensor(token_idxes, max_len=100, padding=True):
    if len(token_idxes) > max_len:
        token_idxes = token_idxes[:max_len]
    n_tokens = len(token_idxes)
    if padding:
        token_idxes = token_idxes \
            + [0] * (max_len - len(token_idxes))
    return torch.tensor(token_idxes, dtype=torch.int64), n_tokens



In [0]:
import torch
from torch import nn, optim
from torch.utils.data import (Dataset, 
                              DataLoader,
                              TensorDataset)
import tqdm


In [0]:
class IMDBDataset(Dataset):
    def __init__(self, dir_path, train=True,
                 max_len=100, padding=True):
        self.max_len = max_len
        self.padding = padding
        
        path = pathlib.Path(dir_path)
        vocab_path = path.joinpath("imdb.vocab")
        
        # 용어집 파일을 읽어서 행 단위로 분할
        self.vocab_array = vocab_path.open() \
                            .read().strip().splitlines()
        # 단어가 키이고 값이 ID인 dict 만들기
        self.vocab_dict = dict((w, i+1) \
            for (i, w) in enumerate(self.vocab_array))
        
        if train:
            target_path = path.joinpath("train")
        else:
            target_path = path.joinpath("test")
        pos_files = sorted(glob.glob(
            str(target_path.joinpath("pos/*.txt"))))
        neg_files = sorted(glob.glob(
            str(target_path.joinpath("neg/*.txt"))))
        # pos는 1, neg는 0인 label을 붙여서
        # (file_path, label)의 튜플 리스트 작성
        self.labeled_files = \
            list(zip([0]*len(neg_files), neg_files )) + \
            list(zip([1]*len(pos_files), pos_files))
    
    @property
    def vocab_size(self):
        return len(self.vocab_array)
    
    def __len__(self):
        return len(self.labeled_files)
    
    def __getitem__(self, idx):
        label, f = self.labeled_files[idx]
        # 파일의 텍스트 데이터를 읽어서 소문자로 변환
        data = open(f).read().lower()
        # 텍스트 데이터를 ID 리스트로 변환
        data = text2ids(data, self.vocab_dict)
        # ID 리스트를 Tensor로 변환
        data, n_tokens = list2tensor(data, self.max_len, self.padding)
        return data, label, n_tokens


In [0]:
train_data = IMDBDataset("/content/aclImdb/")
test_data = IMDBDataset("/content/aclImdb/", train=False)
train_loader = DataLoader(train_data, batch_size=32,
                          shuffle=True, num_workers=4)
test_loader = DataLoader(test_data, batch_size=32,
                        shuffle=False, num_workers=4)


In [0]:
class SequenceTaggingNet(nn.Module):
    def __init__(self, num_embeddings,
                 embedding_dim=50, 
                 hidden_size=50,
                 num_layers=1,
                 dropout=0.2):
        super().__init__()
        self.emb = nn.Embedding(num_embeddings, embedding_dim,
                            padding_idx=0)
        self.lstm = nn.LSTM(embedding_dim,
                            hidden_size, num_layers,
                            batch_first=True, dropout=dropout)
        self.linear = nn.Linear(hidden_size, 1)


        
        
    def forward(self, x, h0=None, l=None):
        # ID를 Embedding으로 다차원 벡터로 변환
        # x는 (batch_size, step_size) 
        # -> (batch_size, step_size, embedding_dim)
        x = self.emb(x)
        # 초기 상태 h0와 함께 RNN에 x를 전달
        # x는(batch_size, step_size, embedding_dim)
        # -> (batch_size, step_size, hidden_dim)
        x, h = self.lstm(x, h0)
        # 마지막 단계만 추출
        # xは(batch_size, step_size, hidden_dim)
        # -> (batch_size, 1)
        if l is not None:
            # 입력의 원래 길이가 있으면 그것을 이용
            x = x[list(range(len(x))), l-1, :]
        else:
            # 없으면 단순히 마지막 것을 이용
            x = x[:, -1, :]
        # 추출한 마지막 단계를 선형 계층에 넣는다
        x = self.linear(x)
        # 불필요한 차원을 삭제
        # (batch_size, 1) -> (batch_size, )
        x = x.squeeze()
        return x


In [0]:
def eval_net(net, data_loader, device="cpu"):
    net.eval()
    ys = []
    ypreds = []
    for x, y, l in data_loader:
        x = x.to(device)
        y = y.to(device)
        l = l.to(device)
        with torch.no_grad():
            y_pred = net(x, l=l)
            y_pred = (y_pred > 0).long()
            ys.append(y)
            ypreds.append(y_pred)
    ys = torch.cat(ys)
    ypreds = torch.cat(ypreds)
    acc = (ys == ypreds).float().sum() / len(ys)
    return acc.item()


In [0]:
from statistics import mean

# num_embeddings에는 0을 포함해서 train_data.vocab_size+1를 넣는다
net = SequenceTaggingNet(train_data.vocab_size+1, 
num_layers=2)
net.to("cuda:0")
opt = optim.Adam(net.parameters())
loss_f = nn.BCEWithLogitsLoss()

for epoch in range(10):
    losses = []
    net.train()
    for x, y, l in tqdm.tqdm(train_loader):
        x = x.to("cuda:0")
        y = y.to("cuda:0")
        l = l.to("cuda:0")
        y_pred = net(x, l=l)
        loss = loss_f(y_pred, y.float())
        net.zero_grad()
        loss.backward()
        opt.step()
        losses.append(loss.item())
    train_acc = eval_net(net, train_loader, "cuda:0")
    val_acc = eval_net(net, test_loader, "cuda:0")
    print(epoch, mean(losses), train_acc, val_acc)


In [0]:
from sklearn.datasets import load_svmlight_file
from sklearn.linear_model import LogisticRegression

train_X, train_y = load_svmlight_file(
    "/content/aclImdb/train/labeledBow.feat")
test_X, test_y = load_svmlight_file(
    "/content/aclImdb/test/labeledBow.feat",
    n_features=train_X.shape[1])

model = LogisticRegression(C=0.1, max_iter=1000)
model.fit(train_X, train_y)
model.score(train_X, train_y), model.score(test_X, test_y)


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


(0.89876, 0.39608)

In [0]:
class SequenceTaggingNet2(SequenceTaggingNet):
    def forward(self, x, h0=None, l=None):
        # ID를 Embedding으로 다차원 벡터로 변환
        x = self.emb(x)
        
        # 길이가 주어진 경우 PckedSequence 만들기
        if l is not None:
            x = nn.utils.rnn.pack_padded_sequence(
                x, l, batch_first=True)
        
        # RNN에 입력
        x, h = self.lstm(x, h0)
        
        # 마지막 단계를 추출해서 선형 계층에 넣는다
        if l is not None:
            # 길이 정보가 있으면 마지막 계층의
            # 내부 상태 벡터를 직접 이용할 수 있다
            # LSTM는 보통 내부 상태 외에 블럭 셀 상태도
            # 가지고 있으므로 내부 상태만 사용한다
            hidden_state, cell_state = h
            x = hidden_state[-1]
        else:
            x = x[:, -1, :]
        
        # 선형 계층에 넣는다
        x = self.linear(x).squeeze()
        return x


In [0]:
for epoch in range(10):
    losses = []
    net.train()
    for x, y, l in tqdm.tqdm(train_loader):
        # 길이 배열을 길이 순으로 정렬
        l, sort_idx = torch.sort(l, descending=True)
        # 얻은 인덱스를 사용해서 x,y도 정렬
        x = x[sort_idx]
        y = y[sort_idx]
        
        x = x.to("cuda:0")
        y = y.to("cuda:0")
        
        y_pred = net(x, l=l)
        loss = loss_f(y_pred, y.float())
        net.zero_grad()
        loss.backward()
        opt.step()
        losses.append(loss.item())
    train_acc = eval_net(net, train_loader, "cuda:0")
    val_acc = eval_net(net, test_loader, "cuda:0")
    print(epoch, mean(losses), train_acc, val_acc)


In [0]:
# 모든 ascii 문자로 사전 만들기
import string
all_chars = string.printable

vocab_size = len(all_chars)
vocab_dict = dict((c, i) for (i, c) in enumerate(all_chars))

# 문자열을 수치 리스트로 변환하는 함수
def str2ints(s, vocab_dict):
    return [vocab_dict[c] for c in s]
  
# 수치 리스트를 문자열로 변환하는 함수
def ints2str(x, vocab_array):
    return "".join([vocab_array[i] for i in x])


In [0]:
from google.colab import files
# 창이 뜨면 파일을 선택해서 업로드한다
uploaded = files.upload()


Saving tinyshakespeare.txt to tinyshakespeare.txt


In [0]:
import torch
from torch import nn, optim
from torch.utils.data import (Dataset, 
                           DataLoader,
                           TensorDataset)
import tqdm

In [0]:
class ShakespeareDataset(Dataset):
    def __init__(self, path, chunk_size=200):
        # 파일을 읽어서 수치 리스트로 변환
        data = str2ints(open(path).read().strip(), vocab_dict)
        
        # Tensor로 변환해서 split 한다
        data = torch.tensor(data, dtype=torch.int64).split(chunk_size)
        
        # 마지막 덩어리(chunk)의 길이를 확인해서 부족한 경우 버린다後のchunkの長さをチェックして足りない場合には捨てる
        if len(data[-1]) < chunk_size:
            data = data[:-1]
        
        self.data = data
        self.n_chunks = len(self.data)
    
    def __len__(self):
        return self.n_chunks

    def __getitem__(self, idx):
        return self.data[idx]


In [0]:
ds = ShakespeareDataset("/content/tinyshakespeare.txt",  chunk_size=200)
loader = DataLoader(ds, batch_size=32, shuffle=True, num_workers=4)


In [0]:
class SequenceGenerationNet(nn.Module):
    def __init__(self, num_embeddings, 
                 embedding_dim=50, 
                 hidden_size=50,
                 num_layers=1, dropout=0.2):
        super().__init__()
        self.emb = nn.Embedding(num_embeddings, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, 
                            hidden_size,
                            num_layers,
                            batch_first=True,
                            dropout=dropout)
        # Linear의 output 크기는 첫 Embedding의 
        # input 크기와 같은 num_embeddings
        self.linear = nn.Linear(hidden_size, num_embeddings)
        
    def forward(self, x, h0=None):
        x = self.emb(x)
        x, h = self.lstm(x, h0)
        x = self.linear(x)
        return x, h


In [0]:
def generate_seq(net, start_phrase="The King said ",
                 length=200, temperature=0.8, device="cpu"):
    # 모델을 평가 모드로 설정
    net.eval()
    # 출력 수치를 저장할 리스트
    result = []
    
    # 시작 문자열을 Tensor로 변환
    start_tensor = torch.tensor(
        str2ints(start_phrase, vocab_dict),
        dtype=torch.int64
    ).to(device)
    # 선두에 batch 차원을 붙인다
    x0 = start_tensor.unsqueeze(0) 
    # RNN을 통해서 출력과 새로운 내부 상태를 얻는다
    o, h = net(x0)
    # 출력을 정규화돼있지 않은 확률로 변환
    out_dist = o[:, -1].view(-1).exp()
    # 확률로부터 실제 문자의 인덱스를 샘플링グ
    top_i = torch.multinomial(out_dist, 1)[0]
    # 결과 저장
    result.append(top_i)
    
    # 생성된 결과를 차례로 RNN에 넣는다
    for i in range(length):
        inp = torch.tensor([[top_i]], dtype=torch.int64)
        inp = inp.to(device)
        o, h = net(inp, h)
        out_dist = o.view(-1).exp()
        top_i = torch.multinomial(out_dist, 1)[0]
        result.append(top_i)
    
    # 시작 문자열과 생성된 문자열을 모아서 반환
    return start_phrase + ints2str(result, all_chars)


In [0]:
from statistics import mean
net = SequenceGenerationNet(vocab_size, 20, 50,
                            num_layers=2, dropout=0.1)
net.to("cuda:0")
opt = optim.Adam(net.parameters())
# 다중 식별 문제이므로 SoftmaxCrossEntropyLoss가 손실 함수가 된다
loss_f = nn.CrossEntropyLoss()

for epoch in range(50):
    net.train()
    losses = []
    for data in tqdm.tqdm(loader):
        # x는 처음부터 마지막의 하나 앞 문자까지
        x = data[:, :-1]
        # y는 두 번째부터 마지막 문자까지
        y = data[:, 1:]
        
        x = x.to("cuda:0")
        y = y.to("cuda:0")
        
        y_pred, _ = net(x)
        # batch와 step 축을 통합해서 CrossEntropyLoss에 전달
        loss = loss_f(y_pred.view(-1, vocab_size), y.view(-1))
        net.zero_grad()
        loss.backward()
        opt.step()
        losses.append(loss.item())
    # 현재 손실 함수와 생성된 문장 예 표시
    print(epoch, mean(losses))
    with torch.no_grad():
        print(generate_seq(net, device="cuda:0"))


100%|██████████| 175/175 [00:08<00:00, 19.89it/s]


0 3.4874473898751397


  0%|          | 0/175 [00:00<?, ?it/s]

The King said yireehpeade x:Cun aesn  no
h 
Thbe
p :oi ioghEecb dat;tsnoiesG I
wnwr'o e br 'tRtfmsWen:eEdgceolf nsO.wn
ovr toiiwb u ;hy] F
h headTroGcfu y  awA
hti
 Tbmh,eB owaEg anla   cN aa
sef uasE'eeieuoto taswA


100%|██████████| 175/175 [00:08<00:00, 20.73it/s]


...

48 1.6790780305862427


  0%|          | 0/175 [00:00<?, ?it/s]

The King said to the will.

LUCIO:
This paul, where Cloece and marrades shales
To tapeck; bay,
Gods heart; spuck; and say, then hourming it of pleaf it,
And at speak priy.

QUEEN MARGARET:
Amonds ffacence evoy that 


100%|██████████| 175/175 [00:08<00:00, 20.79it/s]


49 1.675383790561131
The King said father: the course a speak generion of thingt wan agay
Blawirg your asporen, Would on me is broughts of forgoor spaicen by
Fair Bamends appeaseth worth;
To unthee one intay to call by I bring, his shou


In [0]:
!wget http://www.manythings.org/anki/spa-eng.zip
!unzip spa-eng.zip


In [0]:
import torch
from torch import nn, optim
from torch.utils.data import (Dataset, 
                              DataLoader,
                              TensorDataset)
import tqdm


In [0]:
import re
import collections
import itertools

remove_marks_regex = re.compile(
    "[\,\(\)\[\]\*:;¿¡]|<.*?>")
shift_marks_regex = re.compile("([?!\.])")

unk = 0
sos = 1
eos = 2

def normalize(text):
    text = text.lower()
    # 불필요한 문자 제거
    text = remove_marks_regex.sub("", text)
    # ?!. 와 단어 사이에 공백 삽입
    text = shift_marks_regex.sub(r" \1", text)
    return text
  
def parse_line(line):
    line = normalize(line.strip())
    # 번역 대상(src)과 번역 결과(trg) 각각의 토큰을 리스트로 만든다
    src, trg = line.split("\t")
    src_tokens = src.strip().split()
    trg_tokens = trg.strip().split()
    return src_tokens, trg_tokens
  
def build_vocab(tokens):
    # 파일 안의 모든 문장에서 토큰의 등장 횟수를 확인
    counts = collections.Counter(tokens)
    # 토큰의 등장 횟수를 많은 순으로 나열
    sorted_counts = sorted(counts.items(), 
                           key=lambda c: c[1], reverse=True)
    # 세 개의 태그를 추가해서 정방향 리스트와 역방향 용어집 만들기
    word_list = ["<UNK>", "<SOS>", "<EOS>"] \
        + [x[0] for x in sorted_counts]
    word_dict = dict((w, i) for i, w in enumerate(word_list))
    return word_list, word_dict
  
def words2tensor(words, word_dict, max_len, padding=0):
    # 끝에 종료 태그를 붙임
    words = words + ["<EOS>"]
    # 사전을 이용해서 수치 리스트로 변환
    words = [word_dict.get(w, 0) for w in words]
    seq_len = len(words)
    # 길이가 max_len이하이면 패딩한다
    if seq_len < max_len + 1:
        words = words + [padding] * (max_len + 1 - seq_len)
    # Tensor로 변환해서 반환
    return torch.tensor(words, dtype=torch.int64), seq_len



In [0]:
class TranslationPairDataset(Dataset):
    def __init__(self, path, max_len=15):
        # 단어 수사 많은 문장을 걸러내는 함수
        def filter_pair(p):
            return not (len(p[0]) > max_len 
                        or len(p[1]) > max_len)
        # 파일을 열어서, 파스 및 필터링       
        with open(path) as fp:
            pairs = map(parse_line, fp)
            pairs = filter(filter_pair, pairs)
            pairs = list(pairs)
        # 문장의 소스와 타켓으로 나눔
        src = [p[0] for p in pairs]
        trg = [p[1] for p in pairs]
        #각각의 어휘집 작성
        self.src_word_list, self.src_word_dict = \
            build_vocab(itertools.chain.from_iterable(src))
        self.trg_word_list, self.trg_word_dict = \
            build_vocab(itertools.chain.from_iterable(trg))
        # 어휘집을 사용해서 Tensor로 변환
        self.src_data = [words2tensor(
            words, self.src_word_dict, max_len)
                for words in src]
        self.trg_data = [words2tensor(
            words, self.trg_word_dict, max_len, -100)
                         for words in trg]
    def __len__(self):
        return len(self.src_data)
      
    def __getitem__(self, idx):
        src, lsrc = self.src_data[idx]
        trg, ltrg = self.trg_data[idx]
        return src, lsrc, trg, ltrg



In [0]:
batch_size = 64
max_len = 10
path = "/content/spa.txt"
ds = TranslationPairDataset(path, max_len=max_len)
loader = DataLoader(ds, batch_size=batch_size, shuffle=True,
                    num_workers=4)


In [0]:
class Encoder(nn.Module):
    def __init__(self, num_embeddings,
                 embedding_dim=50, 
                  hidden_size=50,
                 num_layers=1,
                 dropout=0.2):
        super().__init__()
        self.emb = nn.Embedding(num_embeddings, 
          embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(embedding_dim,
                            hidden_size, num_layers,
                            batch_first=True,
dropout=dropout)
        
    def forward(self, x, h0=None, l=None):
        x = self.emb(x)
        if l is not None:
            x = nn.utils.rnn.pack_padded_sequence(
                x, l, batch_first=True)
        _, h = self.lstm(x, h0)
        return h


In [0]:
class Decoder(nn.Module):
    def __init__(self, num_embeddings,
                 embedding_dim=50, 
                 hidden_size=50,
                 num_layers=1,
                 dropout=0.2):
        super().__init__()
        self.emb = nn.Embedding(num_embeddings, embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(embedding_dim, hidden_size,
                            num_layers, batch_first=True,
                            dropout=dropout)
        self.linear = nn.Linear(hidden_size, num_embeddings)
    
    def forward(self, x, h, l=None):
        x = self.emb(x)
        if l is not None:
            x = nn.utils.rnn.pack_padded_sequence(
                x, l, batch_first=True)
        x, h = self.lstm(x, h)
        if l is not None:
            x = nn.utils.rnn.pad_packed_sequence(x, batch_first=True, padding_value=0)[0]
        x = self.linear(x)
        return x, h


In [0]:
def translate(input_str, enc, dec, max_len=15, device="cpu"):
    # 입력 문자열을 수치화해서 Tensor로 변환
    words = normalize(input_str).split()
    input_tensor, seq_len = words2tensor(words, 
        ds.src_word_dict, max_len=max_len)
    input_tensor = input_tensor.unsqueeze(0)
    # 엔코더에서 사용하므로 입력값의 길이도 리스트로 만들어둔다
    seq_len = [seq_len]
    # 시작 토큰 준비
    sos_inputs = torch.tensor(sos, dtype=torch.int64)
    input_tensor = input_tensor.to(device)
    sos_inputs = sos_inputs.to(device)
    # 입력 문자열을 엔코더에 넣어서 컨텍스트 얻기
    ctx = enc(input_tensor, l=seq_len)
    # 시작 토큰과 컨텍스트를 디코더의 초깃값으로 설정
    z = sos_inputs
    h = ctx
    results = []
    for i in range(max_len):
        # Decoder로 다음 단어 예측
        o, h = dec(z.view(1, 1), h)
        # 선형 계층의 출력이 가장 큰 위치가 다음 단어의 ID
        wi = o.detach().view(-1).max(0)[1]
        if wi.item() == eos:
            break
        results.append(wi.item())
        # 다음 입력값으로 현재 출력 ID를 사용
        z = wi
    # 기록해둔 출력 ID를 문자열로 변환
    return " ".join(ds.trg_word_list[i] for i in results)


In [0]:
enc = Encoder(len(ds.src_word_list), 100, 100, 2)
dec = Decoder(len(ds.trg_word_list), 100, 100, 2)
translate("I am a student.", enc, dec)


'susurro liberaron salida trasladé trasladé moscú moscú memorizando moscú moscú moscú moscú memorizando moscú moscú'

In [0]:
enc = Encoder(len(ds.src_word_list), 100, 100, 2)
dec = Decoder(len(ds.trg_word_list), 100, 100, 2)
enc.to("cuda:0")
dec.to("cuda:0")
opt_enc = optim.Adam(enc.parameters(), 0.002)
opt_dec = optim.Adam(dec.parameters(), 0.002)
loss_f = nn.CrossEntropyLoss()


In [0]:
from statistics import mean

def to2D(x):
    shapes = x.shape
    return x.reshape(shapes[0] * shapes[1], -1)
  
for epoc in range(30):
    # 신경망을 훈련 모드로 설정
    enc.train(), dec.train()
    losses = []
    for x, lx, y, ly in tqdm.tqdm(loader):
        # x의 PackedSequence를 만들기 위해 번역 소스의 길이로 내림차순 정렬한다
        lx, sort_idx = lx.sort(descending=True)
        x, y, ly = x[sort_idx], y[sort_idx], ly[sort_idx]
        x, y = x.to("cuda:0"), y.to("cuda:0")
        # 번역 소스를 엔코더에 넣어서 컨텍스트를 얻는다
        ctx = enc(x, l=lx)
        # y의 PackedSequence를 만들기 위해 번역 소스의 길이로 내림차순 정렬
        ly, sort_idx = ly.sort(descending=True)
        y = y[sort_idx]
        # Decoder의 초깃값 설정
        h0 = (ctx[0][:, sort_idx, :], ctx[1][:, sort_idx, :])
        z = y[:, :-1].detach()
        # -100인 상태에선 Embedding 계산에서 오류가 발생하므로 0으로 변경
        z[z==-100] = 0
        # 디코더에 넣어서 손실 함수 계산
        o, _ = dec(z, h0, l=ly-1)
        loss = loss_f(to2D(o[:]), to2D(y[:, 1:max(ly)]).squeeze())
        # Backpropagation(오차 역전파 실행)
        enc.zero_grad(), dec.zero_grad()
        loss.backward()
        opt_enc.step(), opt_dec.step()
        losses.append(loss.item())
    # 전체 데이터의 계산이 끝나면 현재의
    # 손실 함수 값이나 번역 결과를 표시
    enc.eval(), dec.eval()
    print(epoc, mean(losses))
    with torch.no_grad():
        print(translate("I am a student.",
                         enc, dec, max_len=max_len, 
device="cuda:0"))
        print(translate("He likes to eat pizza.",
                         enc, dec, max_len=max_len, 
device="cuda:0"))
        print(translate("She is my mother.",
                         enc, dec, max_len=max_len, 
device="cuda:0"))


  0%|          | 0/1623 [00:00<?, ?it/s]

0 5.343641384545477
un poco .
que mary estaba .
es mi padre .


  0%|          | 0/1623 [00:00<?, ?it/s]

1 3.357381745759749
un niño .
que él estaba equivocado .
es mi madre .


  0%|          | 0/1623 [00:00<?, ?it/s]

...

  0%|          | 0/1623 [00:00<?, ?it/s]

18 0.5737439397302441
un estudiante .
que él se estudia con facilidad .
mi madre .


  0%|          | 0/1623 [00:00<?, ?it/s]

19 0.556303559228071
un estudiante .
que nos gusta aprender .
mi madre .


 74%|███████▍  | 1208/1623 [01:00<00:20, 20.08it/s]